In [1]:
import os
import pickle
from data_generator import DataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model
from tensorflow import keras

In [2]:
image_size = (64, 64)
image_channels = 1
n_classes = 4

folder_name = 'processed-1'
partitions = pickle.load(open('meta/%s/partitions.bin' % folder_name, 'rb'))
labels = pickle.load(open('meta/%s/labels.bin' % folder_name, 'rb'))

# Model

In [4]:
model = keras.Sequential([
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2), strides=2),
    BatchNormalization(),
    
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    BatchNormalization(),
    
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    BatchNormalization(),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax')
])

model.compile(
    optimizer = Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [5]:
history = model.fit(
    DataGenerator(
        list_IDs = partitions['train'],
        labels = labels,
        batch_size = 64,
        dim = image_size, 
        n_channels = image_channels,
        n_classes = n_classes,
        shuffle = True,
        directory = 'data/%s/' % folder_name
    ),
    validation_data = DataGenerator(
        list_IDs = partitions['validation'],
        labels = labels,
        batch_size = 1,
        dim = image_size, 
        n_channels = image_channels,
        n_classes = n_classes,
        shuffle = False,
        directory = 'data/%s/' % folder_name
    ),
    epochs = 5
)

Epoch 1/5
1500/1500 [==============================] - 942s 628ms/step - loss: 0.2698 - accuracy: 0.9232 - val_loss: 0.1805 - val_accuracy: 0.9465
Epoch 2/5
1500/1500 [==============================] - 911s 608ms/step - loss: 0.1967 - accuracy: 0.9425 - val_loss: 0.1868 - val_accuracy: 0.9486
Epoch 3/5
1500/1500 [==============================] - 902s 601ms/step - loss: 0.1798 - accuracy: 0.9470 - val_loss: 0.1602 - val_accuracy: 0.9531
Epoch 4/5
1500/1500 [==============================] - 907s 604ms/step - loss: 0.1667 - accuracy: 0.9497 - val_loss: 0.1650 - val_accuracy: 0.9528
Epoch 5/5
1500/1500 [==============================] - 810s 540ms/step - loss: 0.1548 - accuracy: 0.9535 - val_loss: 0.1655 - val_accuracy: 0.9530


In [6]:
model.save('models/002_5eps.h5')